In [7]:
import asyncio
import aiohttp
import aiodns
import pandas as pd
import socket
import os
from pathlib import Path
from urllib.parse import urlparse
from tqdm.asyncio import tqdm_asyncio
from tqdm import tqdm
import csv
import ssl

# CONFIG
FOLDER = Path("C:\\Users\\Turtleneck-4\\Desktop\\Phishing_Detection\\models\\model_training\\PhishingUrlData")   # folder with Excel files
EXCEL_GLOB = "*.xls*"
SHEET_NAME_OUT = "Active_Domains"
POSSIBLE_COLUMN_NAMES = [
    "Identified Phishing/Suspected Domain Name",
    "Identified Phishing/Suspected Domain",
    "Domain",
    "URL",
    "Identified Phishing/Suspected Domain Name ",
    "Identified Phishing/Suspected Domain Name  ",
    "Phishing/Suspected Domain Name",
    "Identified Phishing/Suspected Domain",
    "Identified Phishing/Suspected Domain Name"
]
MAX_CONCURRENT = 40            # concurrency for HTTP checks (reduce if hitting issues)
REQUEST_TIMEOUT = 10          # seconds for HTTP request timeout
DNS_TIMEOUT = 5               # seconds for DNS lookup
FOLLOW_REDIRECTS = True
VERIFY_SSL = True             # set False to accept invalid SSL/TLS (not recommended)
SUMMARY_CSV = "checker_summary.csv"

# Helper to normalise extracted candidate
def normalize_value(v):
    if pd.isna(v):
        return None
    s = str(v).strip()
    if not s:
        return None
    return s

def guess_url_candidates_from_cell(cell_text):
    # return a list of url candidates in that cell (usually one)
    s = cell_text.strip()
    # if it looks like just a domain (no scheme), return as-is
    return [s]

# DNS resolve (uses aiodns)
async def resolve_domain(resolver, domain):
    try:
        # try A record
        ans = await resolver.gethostbyname(domain, socket.AF_INET)
        if ans and ans.addresses:
            return True, ans.addresses
        return False, []
    except Exception:
        return False, []

# Try HTTP(S) request attempts
async def try_http(session, candidate):
    """
    Try the candidate as-is and with http/https prefixes. Returns (status_ok, details)
    details: dict with keys: 'final_url','status','error'
    """
    tried = []
    to_try = []
    parsed = urlparse(candidate)
    if parsed.scheme:
        to_try.append(candidate)
    else:
        # try as-is (may be domain-only) then http and https
        to_try.append(candidate)
        to_try.append("https://" + candidate)
        to_try.append("http://" + candidate)

    for url in to_try:
        if url in tried:
            continue
        tried.append(url)
        try:
            # Make a HEAD first to be light, fallback to GET if server rejects HEAD
            timeout = aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
            async with session.head(url, allow_redirects=FOLLOW_REDIRECTS, timeout=timeout) as resp:
                status = resp.status
                final = str(resp.url)
                if 200 <= status < 400:
                    return True, {"final_url": final, "status": status, "tried": url}
                # treat some 4xx/5xx as not active
                # continue trying other schemes
        except aiohttp.ClientResponseError as e:
            # maybe HEAD not allowed, try GET
            try:
                timeout = aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
                async with session.get(url, allow_redirects=FOLLOW_REDIRECTS, timeout=timeout) as resp2:
                    status = resp2.status
                    final = str(resp2.url)
                    if 200 <= status < 400:
                        return True, {"final_url": final, "status": status, "tried": url}
            except Exception:
                pass
        except Exception:
            # network error, continue to next variant
            pass
    return False, {"error": "all attempts failed", "tried": tried}

# Main check per cell/value
async def check_candidate(session, resolver, raw_value, semaphore):
    """
    returns dict:
      {
        'raw_value': raw_value,
        'normalized': normalized_value,
        'dns_resolved': True/False,
        'http_active': True/False,
        'http_details': {...},
        'active': True/False
      }
    """
    out = {"raw_value": raw_value}
    val = normalize_value(raw_value)
    out['normalized'] = val
    if not val:
        out.update({'dns_resolved': False, 'http_active': False, 'http_details': None, 'active': False})
        return out

    # extract domain part for DNS - if it's a URL, parse
    parsed = urlparse(val if "://" in val else ("http://" + val))
    domain = parsed.hostname

    dns_ok = False
    dns_addrs = []
    if domain:
        try:
            # DNS resolve
            dns_ok, dns_addrs = await resolve_domain(resolver, domain)
        except Exception:
            dns_ok = False

    out['dns_resolved'] = dns_ok
    out['dns_addresses'] = dns_addrs

    # HTTP check (preferred indicator)
    # Use semaphore to limit concurrency
    async with semaphore:
        try:
            http_ok, http_details = await try_http(session, val)
        except Exception as e:
            http_ok = False
            http_details = {"error": str(e)}

    out['http_active'] = http_ok
    out['http_details'] = http_details

    # decide active: prefer http_ok; if http_ok False but DNS ok treat as active=maybe (parked)
    if http_ok:
        out['active'] = True
    else:
        # if DNS resolves but no HTTP, it could be a parked domain or non-web host; treat as inactive for web-case
        out['active'] = False

    return out

async def process_file(path: Path, summary_rows):
    print(f"Processing: {path}")
    # read Excel file
    try:
        xls = pd.ExcelFile(path)
    except Exception as e:
        print(f"  ERROR reading {path}: {e}")
        return

    # For simplicity, use first sheet to find domain column
    df_all = pd.read_excel(path, sheet_name=0, engine="openpyxl")
    # find column containing domains
    domain_col = None
    for cname in POSSIBLE_COLUMN_NAMES:
        if cname in df_all.columns:
            domain_col = cname
            break
    if domain_col is None:
        # fallback: find first column whose dtype is object and contains something with dot
        for col in df_all.columns:
            sample = df_all[col].dropna().astype(str).head(50).tolist()
            if any("." in s for s in sample):
                domain_col = col
                break
    if domain_col is None:
        # give up for this file
        print(f"  Could not auto-detect domain/URL column in {path.name}. Skipping.")
        return

    print(f"  Using column: {domain_col}")

    # prepare aiohttp client + dns resolver
    conn = aiohttp.TCPConnector(ssl=ssl.create_default_context() if VERIFY_SSL else False, limit_per_host=MAX_CONCURRENT)
    resolver = aiodns.DNSResolver(timeout=DNS_TIMEOUT)
    timeout = aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)

    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        tasks = []
        rows = []
        for idx, row in df_all.iterrows():
            rawval = row.get(domain_col, None)
            tasks.append(check_candidate(session, resolver, rawval, semaphore))
            rows.append((idx, row))  # keep row index and row

        results = []
        # tqdm wrapper for async tasks
        for coro in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc=f"  Checking {path.name}"):
            res = await coro
            results.append(res)

    # Map results back to rows order (as they came)
    active_indices = []
    active_records = []
    for (idx, row), res in zip(rows, results):
        active_flag = res.get('active', False)
        if active_flag:
            # combine row dict with check details
            rdict = row.to_dict()
            rdict['_checked_normalized'] = res.get('normalized')
            rdict['_dns_resolved'] = res.get('dns_resolved')
            rdict['_dns_addresses'] = ", ".join(res.get('dns_addresses') or [])
            rdict['_http_active'] = res.get('http_active')
            rdict['_http_details'] = str(res.get('http_details'))
            active_records.append(rdict)
            active_indices.append(idx)

        # summary row add
        summary_rows.append({
            "filename": path.name,
            "row_index": idx,
            "raw_value": res.get('raw_value'),
            "normalized": res.get('normalized'),
            "dns_resolved": res.get('dns_resolved'),
            "http_active": res.get('http_active'),
            "http_details": str(res.get('http_details')),
            "active": res.get('active')
        })

    # write active records to new sheet in same workbook
    if active_records:
        out_df = pd.DataFrame(active_records)
        try:
            # open existing workbook, append sheet
            with pd.ExcelWriter(path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
                out_df.to_excel(writer, sheet_name=SHEET_NAME_OUT, index=False)
            print(f"  Wrote {len(active_records)} active rows to sheet '{SHEET_NAME_OUT}' in {path.name}")
        except Exception as e:
            print(f"  ERROR writing sheet to {path.name}: {e}")
    else:
        print(f"  No active rows found in {path.name}; no sheet written.")

async def main():
    folder = Path(FOLDER)
    files = list(folder.glob(EXCEL_GLOB))
    if not files:
        print(f"No Excel files found in {folder.resolve()}")
        return

    summary_rows = []
    # process each file sequentially (to avoid too many concurrent connections across files)
    for f in files:
        try:
            await process_file(f, summary_rows)
        except Exception as e:
            print(f"Error processing {f.name}: {e}")

    # write summary csv
    if summary_rows:
        keys = ["filename","row_index","raw_value","normalized","dns_resolved","http_active","http_details","active"]
        with open(SUMMARY_CSV, "w", newline="", encoding="utf-8") as csvf:
            writer = csv.DictWriter(csvf, fieldnames=keys)
            writer.writeheader()
            writer.writerows(summary_rows)
        print(f"Summary written to {SUMMARY_CSV}")
        df_summary = pd.DataFrame(summary_rows)
        active_df = df_summary[df_summary["active"] == True]

        if not active_df.empty:
            active_file = "ActiveDomains_Final.xlsx"
            active_df.to_excel(active_file, index=False)
            print(f"✅ Saved {len(active_df)} active domains to {active_file}")
        else:
            print("⚠️ No active domains found across all files.")
    

import nest_asyncio
import asyncio

nest_asyncio.apply()
await main()


Processing: C:\Users\Turtleneck-4\Desktop\Phishing_Detection\models\model_training\PhishingUrlData\Mock_Data_01_08_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_01_08_2025.xlsx:   0%|          | 0/1244 [00:00<?, ?it/s]C:\Users\Turtleneck-4\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py:427: RuntimeWarning: coroutine 'main' was never awaited
  return futures.Future(loop=self)
  Checking Mock_Data_01_08_2025.xlsx: 100%|██████████| 1244/1244 [01:20<00:00, 15.50it/s]


  Wrote 402 active rows to sheet 'Active_Domains' in Mock_Data_01_08_2025.xlsx
Processing: C:\Users\Turtleneck-4\Desktop\Phishing_Detection\models\model_training\PhishingUrlData\Mock_Data_03_09_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_03_09_2025.xlsx: 100%|██████████| 1372/1372 [00:42<00:00, 32.59it/s]


  Wrote 1333 active rows to sheet 'Active_Domains' in Mock_Data_03_09_2025.xlsx
Processing: C:\Users\Turtleneck-4\Desktop\Phishing_Detection\models\model_training\PhishingUrlData\Mock_Data_04_08_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


...

  No active rows found in Mock_Data_28_08_2025.xlsx; no sheet written.
Processing: C:\Users\Turtleneck-4\Desktop\Phishing_Detection\models\model_training\PhishingUrlData\Mock_Data_29_08_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_29_08_2025.xlsx: 100%|██████████| 12/12 [00:00<00:00, 21.35it/s]


  No active rows found in Mock_Data_29_08_2025.xlsx; no sheet written.
Processing: C:\Users\Turtleneck-4\Desktop\Phishing_Detection\models\model_training\PhishingUrlData\Mock_Data_31_07_2025.xlsx
  Using column: Identified Phishing/Suspected Domain Name


  Checking Mock_Data_31_07_2025.xlsx: 100%|██████████| 302/302 [00:23<00:00, 12.71it/s]

  Wrote 21 active rows to sheet 'Active_Domains' in Mock_Data_31_07_2025.xlsx
Summary written to checker_summary.csv
✅ Saved 1808 active domains to ActiveDomains_Final.xlsx
